In [1]:
import numpy as np
import pandas as pd

In [2]:
def agg_scores(score_list):
    df_score = pd.DataFrame(score_list, columns = ['team_1', 'team_2', 'score_1', 'score_2'])
    df_score2 = df_score.copy()
    
    columns_titles = ['team_2','team_1', 'score_2', 'score_1']
    df_score2=df_score2.reindex(columns=columns_titles)
    df_score2.rename(columns={'team_2': 'team_1', 'team_1': 'team_2', 'score_2': 'score_1', 'score_1': 'score_2'}, inplace = True)
    
    df_score = df_score.append(df_score2, ignore_index = True)
    df_score['margin'] = df_score['score_1'] - df_score['score_2']
    
    return df_score

In [3]:
def generate_point_diff(df):
    teams = sorted(df_score['team_1'].unique())
    total_teams = len(teams)

    point_differential = np.zeros((total_teams))
    points_for = np.zeros((total_teams))
    points_against = np.zeros((total_teams))
    
    for idx, team in enumerate(teams):
        point_differential[idx] = df.loc[df['team_1'] == team]['margin'].sum()
        points_for[idx] = df.loc[df['team_1'] == team]['score_1'].sum()
        points_against[idx] = df.loc[df['team_1'] == team]['score_2'].sum()

    return point_differential, points_for, points_against

In [4]:
def matrix_M(df):
    total_games = df['team_1'].value_counts().sort_index(ascending=False).sort_values(ascending=False).tolist()
    teams = sorted(df_score['team_1'].unique())
    total_teams = len(teams)
    
    M = np.ones((total_teams, total_teams)) * -1
    row, col = np.diag_indices_from(M)
    M[row,col] = np.array(total_games)
    return M

In [9]:
def calculate_rating(M, point_differential):
    M_ = M.copy()
    p = point_differential.copy()
    M_[4,:] = np.array([1,1,1,1,1])
    p[-1] = 0

    return np.linalg.solve(M_, p) 

In [74]:
def defensive_score(M, r, points_for):
    # Matriz diagonal
    T = np.diag(np.diag(M))
    P = T - M

    a = T + P
    b = T.dot(r) - points_for
    return np.round(np.linalg.solve(a, b), decimals=2)


In [82]:
def ranking(list_to_rank):
    list_size = len(list_to_rank)
    rank = [0]*list_size
    
    for pos, idx in enumerate(np.argsort(list_to_rank)[::-1]):
        rank[idx] = pos + 1

    return rank

In [80]:
def massey_rank(score_list):
    df_score = agg_scores(games_score)
    point_differential, points_for, points_against = generate_point_diff(df_score)
    M = matrix_M(df_score)
    r = calculate_rating(M, point_differential)
    defensive = defensive_score(M, r, points_for)
    ofensive = r - defensive
    
    teams = sorted(df_score['team_1'].unique())
    rank_r = ranking(r)
    rank_ofensive = ranking(ofensive)
    rank_defensive = ranking(defensive)
    
    list_of_tuples = list(zip(teams, r, rank_r, ofensive, rank_ofensive, defensive, rank_defensive))  
    return pd.DataFrame(list_of_tuples, columns = ['Team', 'r', 'Rank with r', 'o', 'Offensive Rank', 'd', 'Dfensive Rank'])    